In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.datasets import imdb
import keras
from keras.utils import to_categorical
from sklearn.metrics import f1_score

BASE_DIR = ''
GLOVE_DIR = '/home/abhinav/data/GLOVE_DATA/glove.6B'
TEXT_DATA_DIR = '/home/abhinav/data/full_text'
THIRD_MOD_DIR = "/home/abhinav/data/third_model/"
SECOND_MOD_DIR = "/home/abhinav/data/second_model/"
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 200
FOURTH_MOD_DIR = "/home/abhinav/data/fourth_model/"

print('Loading data...')
[x_train, y_train] = pd.read_pickle(FOURTH_MOD_DIR + "train_data_10ratio") 

[x_test, y_test] = pd.read_pickle(FOURTH_MOD_DIR + "test_dataset") 

[x_val, y_val ] = pd.read_pickle(FOURTH_MOD_DIR + "validation_dataset")

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D,merge, MaxPooling1D, Embedding,Merge,  Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers.merge import Add,Concatenate,Dot

num_words = MAX_NB_WORDS

embedding_matrix = pd.read_pickle("/home/abhinav/data/fourth_model/embedding_matrix_for_selected_words")

Using TensorFlow backend.


Loading data...


In [2]:
class Metrics(keras.callbacks.Callback):
    def __init__(self,filepath):
        self.filepath = filepath
        self.best = -np.Inf
    def on_epoch_end(self, epoch,batch, logs={}):
        predict = self.model.predict(self.validation_data[0],batch_size = 512)
        predict = prob_to_label(predict)
        targ = self.validation_data[1]
        self.f1s=f1(targ, predict)
        """if self.f1s > self.best:
            self.best = self.f1s
            print('Epoch %05d: improved f1 to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch, self.best, self.filepath))
            self.model.save(self.filepath,overwrite =True)"""
        print ("\nWeighted F1 score found on Validation dataset : " ,self.f1s)
        return

from sklearn.metrics import precision_recall_fscore_support
def f1(y_true, y_pred):
    return precision_recall_fscore_support(y_true = y_true, y_pred = y_pred)
ndm = keras.optimizers.Nadam(lr = 0.001)    

def prob_to_label(array):
    for a in array:
        if a[0] > a[1] :
            a[0] = 1
            a[1] = 0
        elif a[0] < a[1]:
            a[0] = 0
            a[1] = 1
        else:
            ind = np.random.randint(2)
            a[ind]= 1
            a[int(1-ind)] = 0
    return array

In [3]:
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers
from keras.layers import Bidirectional, LSTM, GRU

In [8]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(filters=32,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    l_drop = Dropout(0.25)(l_pool)
    convs.append(l_drop)
    
l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(32, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(10)(l_cov1)
drop1 = Dropout(0.25)(l_pool1)
l_flat = Flatten()(drop1)
l_dense = Dense(128, activation='relu')(l_flat)
drop2 = Dropout(0.5)(l_dense)
preds = Dense(2, activation='softmax')(drop2)


model = Model(sequence_input, preds)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 1000)          0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1000, 200)     10000000    input_2[0][0]                    
____________________________________________________________________________________________________
conv1d_5 (Conv1D)                (None, 998, 32)       19232       embedding_2[0][0]                
____________________________________________________________________________________________________
conv1d_6 (Conv1D)                (None, 997, 32)       25632       embedding_2[0][0]                
___________________________________________________________________________________________

/home/abhinav/miniconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:20: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [11]:
model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

In [12]:
initial_weights = model.get_weights()

In [14]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 0s - loss: 1.0695 - acc: 0.5984
Weighted F1 score found on Validation dataset :  (array([ 0.9975108 ,  0.08698539]), array([ 0.90833333,  0.79393939]), array([ 0.95083569,  0.15679234]), array([15000,   165]))
13150/13150 [==============================] - 253s - loss: 1.0658 - acc: 0.5997 - val_loss: 0.2522 - val_acc: 0.9071
Epoch 2/20
13056/13150 [============================>.] - ETA: 0s - loss: 0.6954 - acc: 0.8268
Weighted F1 score found on Validation dataset :  (array([ 0.99811449,  0.07345226]), array([ 0.88226667,  0.84848485]), array([ 0.93662196,  0.13520039]), array([15000,   165]))
13150/13150 [==============================] - 247s - loss: 0.6942 - acc: 0.8275 - val_loss: 0.2680 - val_acc: 0.8819
Epoch 3/20
13056/13150 [============================>.] - ETA: 0s - loss: 0.5779 - acc: 0.8503
Weighted F1 score found on Validation dataset :  (array([ 0.99850131,  0.

In [15]:
del(model)
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# applying a more complex convolutional approach
convs = []
filter_sizes = [4,5,6]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(filters=32,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    l_drop = Dropout(0.25)(l_pool)
    convs.append(l_drop)
    
l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(32, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(10)(l_cov1)
drop1 = Dropout(0.25)(l_pool1)
l_flat = Flatten()(drop1)
l_dense = Dense(128, activation='relu')(l_flat)
drop2 = Dropout(0.5)(l_dense)
preds = Dense(2, activation='softmax')(drop2)


model = Model(sequence_input, preds)

model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 1000)          0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 1000, 200)     10000000    input_3[0][0]                    
____________________________________________________________________________________________________
conv1d_9 (Conv1D)                (None, 997, 32)       25632       embedding_3[0][0]                
____________________________________________________________________________________________________
conv1d_10 (Conv1D)               (None, 996, 32)       32032       embedding_3[0][0]                
___________________________________________________________________________________________

/home/abhinav/miniconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:21: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [16]:

model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

#model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.5225 - acc: 0.4278
Weighted F1 score found on Validation dataset :  (array([ 0.99776514,  0.02687465]), array([ 0.6548    ,  0.86666667]), array([ 0.79069393,  0.0521327 ]), array([15000,   165]))
13150/13150 [==============================] - 312s - loss: 1.5191 - acc: 0.4294 - val_loss: 0.5386 - val_acc: 0.6571
Epoch 2/20
13056/13150 [============================>.] - ETA: 1s - loss: 0.8405 - acc: 0.7582
Weighted F1 score found on Validation dataset :  (array([ 0.99893955,  0.03975058]), array([ 0.7536    ,  0.92727273]), array([ 0.85909713,  0.07623318]), array([15000,   165]))
13150/13150 [==============================] - 308s - loss: 0.8407 - acc: 0.7592 - val_loss: 0.4916 - val_acc: 0.7555
Epoch 3/20
13056/13150 [============================>.] - ETA: 1s - loss: 0.6659 - acc: 0.8255
Weighted F1 score found on Validation dataset :  (array([ 0.99841358,  0.

KeyboardInterrupt: 

In [ ]:
del(model)
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# applying a more complex convolutional approach
convs = []
filter_sizes = [5,6,7]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(filters=32,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    l_drop = Dropout(0.25)(l_pool)
    convs.append(l_drop)
    
l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(32, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(10)(l_cov1)
drop1 = Dropout(0.25)(l_pool1)
l_flat = Flatten()(drop1)
l_dense = Dense(128, activation='relu')(l_flat)
drop2 = Dropout(0.5)(l_dense)
preds = Dense(2, activation='softmax')(drop2)


model = Model(sequence_input, preds)

model.summary()


In [ ]:

model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

#model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
del(model)
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# applying a more complex convolutional approach
convs = []
filter_sizes = [6,7,8]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(filters=32,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    l_drop = Dropout(0.25)(l_pool)
    convs.append(l_drop)
    
l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(32, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(10)(l_cov1)
drop1 = Dropout(0.25)(l_pool1)
l_flat = Flatten()(drop1)
l_dense = Dense(128, activation='relu')(l_flat)
drop2 = Dropout(0.5)(l_dense)
preds = Dense(2, activation='softmax')(drop2)


model = Model(sequence_input, preds)

print(model.summary())

model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

#model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
del(model)
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# applying a more complex convolutional approach
convs = []
filter_sizes = [8,9,10]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(filters=32,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    l_drop = Dropout(0.25)(l_pool)
    convs.append(l_drop)
    
l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(32, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(10)(l_cov1)
drop1 = Dropout(0.25)(l_pool1)
l_flat = Flatten()(drop1)
l_dense = Dense(128, activation='relu')(l_flat)
drop2 = Dropout(0.5)(l_dense)
preds = Dense(2, activation='softmax')(drop2)


model = Model(sequence_input, preds)

print(model.summary())

model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

#model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [17]:
del(model)
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# applying a more complex convolutional approach
convs = []
filter_sizes = [4,5,6,7,8]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(filters=32,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    l_drop = Dropout(0.25)(l_pool)
    convs.append(l_drop)
    
l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(64, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(10)(l_cov1)
drop1 = Dropout(0.25)(l_pool1)
l_flat = Flatten()(drop1)
l_dense = Dense(128, activation='relu')(l_flat)
drop2 = Dropout(0.5)(l_dense)
preds = Dense(2, activation='softmax')(drop2)


model = Model(sequence_input, preds)

print(model.summary())

model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

#model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

/home/abhinav/miniconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:21: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 1000)          0                                            
____________________________________________________________________________________________________
embedding_4 (Embedding)          (None, 1000, 200)     10000000    input_4[0][0]                    
____________________________________________________________________________________________________
conv1d_13 (Conv1D)               (None, 997, 32)       25632       embedding_4[0][0]                
____________________________________________________________________________________________________
conv1d_14 (Conv1D)               (None, 996, 32)       32032       embedding_4[0][0]                
___________________________________________________________________________________________

13150/13150 [==============================] - 614s - loss: 0.5596 - acc: 0.8515 - val_loss: 0.4710 - val_acc: 0.7499
Epoch 7/20
13056/13150 [============================>.] - ETA: 2s - loss: 0.5218 - acc: 0.8530
Weighted F1 score found on Validation dataset :  (array([ 0.99888624,  0.05818882]), array([ 0.83706667,  0.91515152]), array([ 0.91084512,  0.10942029]), array([15000,   165]))
13150/13150 [==============================] - 603s - loss: 0.5202 - acc: 0.8530 - val_loss: 0.3089 - val_acc: 0.8379
Epoch 8/20
13056/13150 [============================>.] - ETA: 2s - loss: 0.4769 - acc: 0.8671
Weighted F1 score found on Validation dataset :  (array([ 0.99917308,  0.05045573]), array([ 0.80553333,  0.93939394]), array([ 0.89196471,  0.09576769]), array([15000,   165]))
13150/13150 [==============================] - 603s - loss: 0.4762 - acc: 0.8670 - val_loss: 0.3619 - val_acc: 0.8070
Epoch 9/20
13056/13150 [============================>.] - ETA: 2s - loss: 0.4714 - acc: 0.8752
Weigh

In [11]:
#model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.0244 - acc: 0.7000
Weighted F1 score found on Validation dataset :  (array([ 0.99831275,  0.06726246]), array([ 0.8678    ,  0.86666667]), array([ 0.92849246,  0.12483632]), array([15000,   165]))
13150/13150 [==============================] - 368s - loss: 1.0207 - acc: 0.7016 - val_loss: 0.3410 - val_acc: 0.8678
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.6201 - acc: 0.8545
Weighted F1 score found on Validation dataset :  (array([ 0.99915154,  0.04587156]), array([ 0.78506667,  0.93939394]), array([ 0.87926529,  0.08747178]), array([15000,   165]))
13150/13150 [==============================] - 325s - loss: 0.6192 - acc: 0.8541 - val_loss: 0.4175 - val_acc: 0.7867
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.4360 - acc: 0.8955
Weighted F1 score found on Validation dataset :  (array([ 0.99868523,  0.

In [12]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 15},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.0880 - acc: 0.6324
Weighted F1 score found on Validation dataset :  (array([ 0.99876594,  0.04983389]), array([ 0.80933333,  0.90909091]), array([ 0.89412631,  0.09448819]), array([15000,   165]))
13150/13150 [==============================] - 317s - loss: 1.0859 - acc: 0.6333 - val_loss: 0.4298 - val_acc: 0.8104
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.7124 - acc: 0.8229
Weighted F1 score found on Validation dataset :  (array([ 0.99730674,  0.08969867]), array([ 0.9134    ,  0.77575758]), array([ 0.95351103,  0.16080402]), array([15000,   165]))
13150/13150 [==============================] - 316s - loss: 0.7101 - acc: 0.8236 - val_loss: 0.2316 - val_acc: 0.9119
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.5304 - acc: 0.8696
Weighted F1 score found on Validation dataset :  (array([ 0.99853112,  0.

In [13]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 20},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.3914 - acc: 0.3807
Weighted F1 score found on Validation dataset :  (array([ 0.999571  ,  0.02756377]), array([ 0.62133333,  0.97575758]), array([ 0.76632133,  0.05361305]), array([15000,   165]))
13150/13150 [==============================] - 322s - loss: 1.3917 - acc: 0.3814 - val_loss: 0.8202 - val_acc: 0.6252
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.8743 - acc: 0.7926
Weighted F1 score found on Validation dataset :  (array([ 0.99938747,  0.0422799 ]), array([ 0.7614    ,  0.95757576]), array([ 0.86431058,  0.08098411]), array([15000,   165]))
13150/13150 [==============================] - 321s - loss: 0.8744 - acc: 0.7924 - val_loss: 0.5052 - val_acc: 0.7635
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.6378 - acc: 0.8548
Weighted F1 score found on Validation dataset :  (array([ 0.99919692,  0.

In [14]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 30},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.5838 - acc: 0.4036
Weighted F1 score found on Validation dataset :  (array([ 0.99924573,  0.0482524 ]), array([ 0.79486667,  0.94545455]), array([ 0.88541512,  0.09181872]), array([15000,   165]))
13150/13150 [==============================] - 320s - loss: 1.5791 - acc: 0.4064 - val_loss: 0.5419 - val_acc: 0.7965
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.9673 - acc: 0.7640
Weighted F1 score found on Validation dataset :  (array([ 0.99930957,  0.04387926]), array([ 0.77193333,  0.95151515]), array([ 0.87102719,  0.08388993]), array([15000,   165]))
13150/13150 [==============================] - 320s - loss: 0.9671 - acc: 0.7647 - val_loss: 0.5668 - val_acc: 0.7739
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.6484 - acc: 0.8555
Weighted F1 score found on Validation dataset :  (array([ 0.99766679,  0.

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=15,
          class_weight = {0. : 1, 1. : 50},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=15,
          class_weight = {0. : 1, 1. : 75},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 100},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 125},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [17]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 150},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 2.7308 - acc: 0.1568
Weighted F1 score found on Validation dataset :  (array([ 1.        ,  0.01704545]), array([ 0.36566667,  1.        ]), array([ 0.53551379,  0.03351955]), array([15000,   165]))
13150/13150 [==============================] - 321s - loss: 2.7281 - acc: 0.1587 - val_loss: 1.2697 - val_acc: 0.3726
Epoch 2/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.8886 - acc: 0.5123
Weighted F1 score found on Validation dataset :  (array([ 0.99960684,  0.03225806]), array([ 0.678     ,  0.97575758]), array([ 0.80797648,  0.06245151]), array([15000,   165]))
13150/13150 [==============================] - 319s - loss: 1.8832 - acc: 0.5138 - val_loss: 0.8559 - val_acc: 0.6812
Epoch 3/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.3161 - acc: 0.7084
Weighted F1 score found on Validation dataset :  (array([ 0.9993245 ,  0.

KeyboardInterrupt: 

In [16]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 200},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 3.3969 - acc: 0.0895
Weighted F1 score found on Validation dataset :  (array([ 0.        ,  0.01088032]), array([ 0.,  1.]), array([ 0.        ,  0.02152642]), array([15000,   165]))
13150/13150 [==============================] - 328s - loss: 3.3978 - acc: 0.0898 - val_loss: 2.6060 - val_acc: 0.0109
Epoch 2/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.9941 - acc: 0.3591
Weighted F1 score found on Validation dataset :  (array([ 1.        ,  0.01807427]), array([ 0.4024,  1.    ]), array([ 0.57387336,  0.03550678]), array([15000,   165]))
13150/13150 [==============================] - 331s - loss: 1.9972 - acc: 0.3592 - val_loss: 1.7261 - val_acc: 0.4089
Epoch 3/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.2913 - acc: 0.7218
Weighted F1 score found on Validation dataset :  (array([ 0.9995438 ,  0.02516805]), array([ 0.58

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 500},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [15]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 1000},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 8.1149 - acc: 0.0897
Weighted F1 score found on Validation dataset :  (array([ 0.        ,  0.01088032]), array([ 0.,  1.]), array([ 0.        ,  0.02152642]), array([15000,   165]))
13150/13150 [==============================] - 325s - loss: 8.0891 - acc: 0.0895 - val_loss: 3.4302 - val_acc: 0.0109

/home/abhinav/miniconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 2/20
 7168/13150 [===============>..............] - ETA: 86s - loss: 3.4647 - acc: 0.0871

KeyboardInterrupt: 